# Use ChromaDocumentStore with Haystack



## Install dependencies

In [3]:
# Install the Chroma integration, Haystack will come as a dependency
!pip install -U chroma-haystack "huggingface_hub>=0.22.0"

## Indexing Pipeline: preprocess, split and index documents
In this section, we will index documents into a Chroma DB collection by building a Haystack indexing pipeline. Here, we are indexing documents from the [VIM User Manuel](https://vimhelp.org/) into the Haystack `ChromaDocumentStore`.

 We have the `.txt` files for these pages in the examples folder for the `ChromaDocumentStore`, so we are using the [`TextFileToDocument`](https://docs.haystack.deepset.ai/v2.0/docs/textfiletodocument) and [`DocumentWriter`](https://docs.haystack.deepset.ai/v2.0/docs/documentwriter) components to build this indexing pipeline.

In [4]:
# Fetch data files from the Github repo
!curl -sL https://github.com/deepset-ai/haystack-core-integrations/tarball/main -o main.tar
!mkdir main
!tar xf main.tar -C main --strip-components 1
!mv main/integrations/chroma/example/data .

mkdir: main: File exists
mv: rename main/integrations/chroma/example/data to ./data: Directory not empty


In [5]:
import os
from pathlib import Path

from haystack import Pipeline
from haystack.components.converters import TextFileToDocument
from haystack.components.writers import DocumentWriter

from haystack_integrations.document_stores.chroma import ChromaDocumentStore

file_paths = ["data" / Path(name) for name in os.listdir("data")]

# Chroma is used in-memory so we use the same instances in the two pipelines below
document_store = ChromaDocumentStore()

indexing = Pipeline()
indexing.add_component("converter", TextFileToDocument())
indexing.add_component("writer", DocumentWriter(document_store))
indexing.connect("converter", "writer")
indexing.run({"converter": {"sources": file_paths}})


{'writer': {'documents_written': 36}}

## Query Pipeline: build retrieval-augmented generation (RAG) pipelines

Once we have documents in the `ChromaDocumentStore`, we can use the accompanying Chroma retrievers to build a query pipeline. The query pipeline below is a simple retrieval-augmented generation (RAG) pipeline that uses Chroma's [query API](https://docs.trychroma.com/usage-guide#querying-a-collection).

You can change the idnexing pipeline and query pipelines here for embedding search by using one of the [`Haystack Embedders`](https://docs.haystack.deepset.ai/v2.0/docs/embedders) accompanied by the  `ChromaEmbeddingRetriever`.


In this example we are using:
- The `HuggingFaceAPIGenerator` with zephyr-7b-beta. (You will need a Hugging Face token to use this model). You can replace this with any of the other [`Generators`](https://docs.haystack.deepset.ai/v2.0/docs/generators)
- The `PromptBuilder` which holds the prompt template. You can adjust this to a prompt of your choice
- The `ChromaQueryRetriver` which expects a list of queries and retieves the `top_k` most relevant documents from your Chroma collection.

In [6]:
import os
from getpass import getpass

os.environ["HF_API_TOKEN"] = getpass("Enter Hugging Face API key:")

Enter Hugging Face API key: ········


In [18]:
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses.chat_message import ChatMessage

prompt = """
Answer the query based on the provided context.
If the context does not contain the answer, say 'Answer not found'.
Context:
{% for doc in documents %}
  {{ doc.content }}
{% endfor %}
query: {{query}}
Answer:
"""

template = [ChatMessage.from_user(prompt)]
prompt_builder = ChatPromptBuilder(template=template)

llm = HuggingFaceAPIChatGenerator(api_type="serverless_inference_api", api_params={"model": "HuggingFaceH4/zephyr-7b-beta", "provider": "featherless-ai"})
retriever = ChromaQueryTextRetriever(document_store)

querying = Pipeline()
querying.add_component("retriever", retriever)
querying.add_component("prompt_builder", prompt_builder)
querying.add_component("llm", llm)

querying.connect("retriever.documents", "prompt_builder.documents")
querying.connect("prompt_builder", "llm")

ChatPromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


🚅 Components
  - retriever: ChromaQueryTextRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: HuggingFaceAPIChatGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

In [19]:
query = "Should I write documentation for my plugin?"
results = querying.run({"retriever": {"query": query, "top_k": 3}, "prompt_builder": {"query": query}, "llm":{"generation_kwargs": {}}})

In [20]:
print(results["llm"]["replies"][0])

ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='How does myplugin know how to do so.\nThese paragraphs explain how to get Faith.  It fits wrong.  When "New Riders what said that you have an argument version file, so that it should jump forward.  Vim -c \'e $VIMRUNTIME!.  This is command.  Asking hopefully that Vim what you have a complete text.  It is a section of "and" is used by "m".  The Archive name files complain about mobile and Vimc "m"\n\nFirst Page --page.  When you create the number, you specify Vimc "m"\nBabeReturned by Vim C call the Archive coverage for more recently, you can use the Archive files where the script.  With myPlugin.vimc m in Vim !\ntry to curl the user manual.  It\'s also done in "and".\nIt has been begun with the number that writing a manual for the manual.  Maybe case myPlugin.vim is Vimc "m" regarding the manual.  It also works.  The headers carry the Archive.  For "m1".  It is indeed a manual with: >\n\t:to CURL all M.\nD